# Weights and Biases (W&B)와 함께 디퓨전 모델 학습하기

이 노트북에서 우리는 W&B와 함께 디퓨전 모델을 학습하는 방법을 배워볼 것입니다.  
["How diffusion models work"](https://www.deeplearning.ai/short-courses/how-diffusion-models-work/) course 강의의 Lab3 노트북을 사용합니다.  

다룰 내용은 다음과 같습니다:
- training loss와 metric의 로그를 남기기
- 학습 동안 모델로부터 샘플을 추출하여 W&B에 업로드
- 모델 체크포인트를 W&B에 저장

In [ ]:
from types import SimpleNamespace
from pathlib import Path
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from utilities import *

import wandb

W&B의 기능을 제대로 이용하기 위해 계정 생성을 추천합니다.  
여기에서는 익명 옵션을 켜고 학습을 진행합니다.

In [ ]:
wandb.login(anonymous="allow")

## 세팅

In [ ]:
# wandb에 로그로 남을 파라미터들을 저장합니다.
DATA_DIR = Path('./data/')
SAVE_DIR = Path('./data/weights/')
SAVE_DIR.mkdir(exist_ok=True, parents=True)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = SimpleNamespace(
    # 하이퍼파라미터
    num_samples = 30,

    # 디퓨전 하이퍼파라미터
    timesteps = 500,
    beta1 = 1e-4,
    beta2 = 0.02,

    # 네트워크 하이퍼파라미터
    n_feat = 64, # 64 hidden dimension feature
    n_cfeat = 5, # context vector is of size 5
    height = 16, # 16x16 이미지
    
    # 학습 하이퍼파라미터
    batch_size = 100,
    n_epoch = 32,
    lrate = 1e-3,
)

### DDPM noise schedular와 sampler를 셋업합니다. (Diffusion 강의에서와 동일합니다)

- perturb_input: 스케줄 상에 정해진 타임스텝에서 이미지에 노이즈를 추가합니다
- sample_ddpm_context: DDPM sampler를 사용하여 이미지를 생성합니다. 우리는 학습 동안 모델로부터 샘플을 추출하기 위해 잏 함수를 사용하고, 어떻게 학습이 진행되는지 확인할 것입니다

In [ ]:
# ddpm sampler 함수를 셋업합니다
perturb_input, sample_ddpm_context = setup_ddpm(config.beta1, 
                                                config.beta2, 
                                                config.timesteps, 
                                                DEVICE)

In [ ]:
# 모델을 구축합니다
nn_model = ContextUnet(in_channels=3, 
                       n_feat=config.n_feat, 
                       n_cfeat=config.n_cfeat, 
                       height=config.height).to(DEVICE)

In [ ]:
# 데이터셋을 불러오고 옵티마이저를 설정합니다
dataset = CustomDataset.from_np(path=DATA_DIR)
dataloader = DataLoader(dataset, 
                        batch_size=config.batch_size, 
                        shuffle=True)
optim = torch.optim.Adam(nn_model.parameters(), lr=config.lrate)

## 학습

각 클래스에서 6개의 샘플과 함께 고정된 context vector를 고릅니다.

In [ ]:
# Noise vecotr
# x_T ~ N(0, 1), sample initial noise
noises = torch.randn(config.num_samples, 3, 
                     config.height, config.height).to(DEVICE)  

# 고정된 context vector는 다음으로부터 추출합니다
ctx_vector = F.one_hot(torch.tensor([0,0,0,0,0,0,   # hero
                                     1,1,1,1,1,1,   # non-hero
                                     2,2,2,2,2,2,   # food
                                     3,3,3,3,3,3,   # spell
                                     4,4,4,4,4,4]), # side-facing 
                       5).to(DEVICE).float()

다음 학습 셀은 CPU로 실행하면 굉장히 오래 걸립니다.  
강의에서는 GPU로 학습된 결과를 보여줍니다.

### 학습 결과는 >> [여기](https://wandb.ai/dlai-course/dlai_sprite_diffusion/runs/pzs3gsyo) << 에서 확인 가능합니다

In [ ]:
# wandb run을 생성합니다
run = wandb.init(project="dlai_sprite_diffusion", 
                 job_type="train", 
                 config=config)

# W&B로부터 config를 가져와 전달합니다
config = wandb.config

for ep in tqdm(range(config.n_epoch), leave=True, total=config.n_epoch):
    # 학습 모드 설정
    nn_model.train()
    optim.param_groups[0]['lr'] = config.lrate*(1-ep/config.n_epoch)
    
    pbar = tqdm(dataloader, leave=False)
    for x, c in pbar:   # x: images  c: context
        optim.zero_grad()
        x = x.to(DEVICE)
        c = c.to(DEVICE)   
        context_mask = torch.bernoulli(torch.zeros(c.shape[0]) + 0.8).to(DEVICE)
        c = c * context_mask.unsqueeze(-1)        
        noise = torch.randn_like(x)
        t = torch.randint(1, config.timesteps + 1, (x.shape[0],)).to(DEVICE) 
        x_pert = perturb_input(x, t, noise)      
        pred_noise = nn_model(x_pert, t / config.timesteps, c=c)      
        loss = F.mse_loss(pred_noise, noise)
        loss.backward()    
        optim.step()

        wandb.log({"loss": loss.item(),
                   "lr": optim.param_groups[0]['lr'],
                   "epoch": ep})


    # 모델을 주기적으로 저장합니다
    if ep%4==0 or ep == int(config.n_epoch-1):
        nn_model.eval()
        ckpt_file = SAVE_DIR/f"context_model.pth"
        torch.save(nn_model.state_dict(), ckpt_file)

        artifact_name = f"{wandb.run.id}_context_model"
        at = wandb.Artifact(artifact_name, type="model")
        at.add_file(ckpt_file)
        wandb.log_artifact(at, aliases=[f"epoch_{ep}"])

        samples, _ = sample_ddpm_context(nn_model, 
                                         noises, 
                                         ctx_vector[:config.num_samples])
        wandb.log({
            "train_samples": [
                wandb.Image(img) for img in samples.split(1)
            ]})
        
# W&B run을 종료합니다
wandb.finish()

학습이 종료된 후 대시보드를 들어가보면 추출된 샘플 이미지들을 학습 단계별로 확인할 수 있습니다.  
학습이 진행됨에 따라 모델이 점점 더 정확한 이미지를 생성하는 것을 알 수 있습니다.